# Test for converting a CNN to QCNN

code is from Pytorch official [tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re
import sys
from functions import *
from convert_to_quaternion import *

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


Please remember every layer's number of parameters except the output features of the classifier must be divisibile by 4

In [18]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(4, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# run on gpu if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net().to(device)
grayscale = transforms.Grayscale(num_output_channels=1)

In [ ]:
# converting normal net to quaternion
quatnet = convert_to_quaternion(Net()).to(device)

## Real valued NN

In [19]:
criterion_real = nn.CrossEntropyLoss()
optimizer_real = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        gray_image = grayscale(inputs)
        # inputs' channels must be divisible by 4 in a QNN,
        # here used to compare results
        inputs = torch.cat([inputs, gray_image],1)
        # zero the parameter gradients
        optimizer_real.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.to(device))
        loss_real = criterion_real(outputs, labels.to(device))
        loss_real.backward()
        optimizer_real.step()

        # print statistics
        running_loss += loss_real.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.247
[1,  4000] loss: 1.871
[1,  6000] loss: 1.703
[1,  8000] loss: 1.565
[1, 10000] loss: 1.515
[1, 12000] loss: 1.454
[2,  2000] loss: 1.390
[2,  4000] loss: 1.345
[2,  6000] loss: 1.322
[2,  8000] loss: 1.283
[2, 10000] loss: 1.270
[2, 12000] loss: 1.270
Finished Training


## Quaternion NN

In [20]:
criterion_quat = nn.CrossEntropyLoss()
optimizer_quat = optim.SGD(quatnet.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss_quat = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        gray_image = grayscale(inputs)
        inputs = torch.cat([inputs, gray_image],1)
        # zero the parameter gradients
        optimizer_quat.zero_grad()

        # forward + backward + optimize
        outputs = quatnet(inputs.to(device))
        loss_quat = criterion_quat(outputs, labels.to(device))
        loss_quat.backward()
        optimizer_quat.step()

        # print statistics
        running_loss += loss_quat.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.384
[1,  4000] loss: 1.737
[1,  6000] loss: 1.621
[1,  8000] loss: 1.536
[1, 10000] loss: 1.467
[1, 12000] loss: 1.436
[2,  2000] loss: 1.707
[2,  4000] loss: 1.334
[2,  6000] loss: 1.319
[2,  8000] loss: 1.265
[2, 10000] loss: 1.263
[2, 12000] loss: 1.228
Finished Training
